In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

In [ ]:
mnist_data, mnist_info = tfds.load(name='mnist',with_info=True,as_supervised=True)
number_validation_sample = tf.cast(0.1*mnist_info.splits['train'].num_examples,tf.int64)
number_test_sample = mnist_info.splits['test'].num_examples

def scale(image, label):
    image = tf.cast(image,tf.float32)
    image /= 255.
    return image,label
scaled_train_and_validated_data = mnist_data['train'].map(scale)
scaled_test_data = mnist_data['test'].map(scale)

buffer_size = 10000
shuffle_scale_train_and_validated_data = scaled_train_and_validated_data.shuffle(buffer_size)
validate_data = shuffle_scale_train_and_validated_data.take(number_validation_sample)
train_data = shuffle_scale_train_and_validated_data.skip(number_validation_sample)

batch_size = 100
train_data = train_data.batch(batch_size)
validate_data = validate_data.batch(number_validation_sample)
test_data = scaled_test_data.batch(number_test_sample)

validation_inputs,validation_targets = next(iter(validate_data))

In [ ]:
input_size = 28*28
output_size = 10
hidden_layer_size = 100
max_number_epoch = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                            tf.keras.layers.Dense(output_size,activation='softmax'),
                            ])
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

In [ ]:
model.fit(train_data,
         epochs = max_number_epoch,
         callbacks=[early_stopping], # early stopping
         validation_data=(validation_inputs,validation_targets),
         verbose=2
         )

In [ ]:
#note that this time the train, validation and test data are not iterable
model.fit(train_inputs, # train inputs
          train_targets, # train targets
          batch_size=batch_size, # batch size
          epochs=max_epochs, # epochs that we will train for (assuming early stopping doesn't kick in)
          # callbacks are functions called by a task when a task is completed
          # task here is to check if val_loss is increasing
          callbacks=[early_stopping], # early stopping
          validation_data=(validation_inputs, validation_targets), # validation data
          verbose = 2 # making sure we get enough information about the training process
          )  

In [ ]:
test_loss,test_accuracy = model.evaluate(test_data)
print('Test Loss = {:.2f}, Test Accuracy = {:.2f}'.format(test_loss,test_accuracy))

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
 
image_path="7.png"
im = image.load_img(image_path, target_size=(28, 28), color_mode = "grayscale")
img = image.img_to_array(im)
img = tf.expand_dims(img, axis=0)      
img = scale(img, '')[0]
prediction_indexes = (model.predict(img)[0])
np.argmax(prediction_indexes, axis=-1)

In [ ]:
img